In [136]:
# Let's get all the PDFs eh! This takes ages - it's about 500mb for each period!

import requests
from bs4 import BeautifulSoup

pdfs = []

urls = {
    "http://www.finance.gov.au/publications/parliamentarians-reporting/parliamentarians-expenditure-P35/":"35",
    'http://www.finance.gov.au/publications/parliamentarians-reporting/parliamentarians-expenditure-P34/':"34",
    'http://www.finance.gov.au/publications/parliamentarians-reporting/parliamentarians_expenditure_P33.html':"33",
    'http://www.finance.gov.au/publications/parliamentarians-reporting/parliamentarians_expenditure_P32.html':"32",
    "http://www.finance.gov.au/publications/parliamentarians-reporting/parliamentarians_expenditure_P31.html":"31",
    "http://www.finance.gov.au/publications/parliamentarians-reporting/parliamentarians_expenditure_T30.html":"30"   
}    

for url in urls.keys():
    rep = requests.get(url)
    soup = BeautifulSoup(rep.content)
    links = soup.find_all("a")
    for l in links:
        try:
            if "pdf" in l.get("href"):
                pdfs.append(l.get("href"))
        except:
            pass



def download_file(url):
    downloaded = !cd pdfs:ls
    local_filename = url.split('/')[-1]
    if "30" in url:
        local_filename = "P30-"+local_filename
    if "31" in url:
        local_filename = "P31-"+local_filename
    if local_filename not in downloaded:
        # NOTE the stream=True parameter
        r = requests.get(url, stream=True)
        with open('pdfs/'+local_filename, 'wb') as f:
            for chunk in r.iter_content(chunk_size=1024): 
                if chunk: # filter out keep-alive new chunks
                    f.write(chunk)
                    #f.flush() commented by recommendation from J.F.Sebastian
        return local_filename
    
    
for pdf in pdfs:
    download_file(pdf)

    

MissingSchema: Invalid URL 'docs/T30/PE-Explanation_Notes.pdf': No schema supplied. Perhaps you meant http://docs/T30/PE-Explanation_Notes.pdf?

In [6]:
import requests
from bs4 import BeautifulSoup

pdfs = []

urls = {
    "http://www.finance.gov.au/publications/parliamentarians-reporting/parliamentarians-expenditure-P35/":"35",
    'http://www.finance.gov.au/publications/parliamentarians-reporting/parliamentarians-expenditure-P34/':"34",
    'http://www.finance.gov.au/publications/parliamentarians-reporting/parliamentarians_expenditure_P33.html':"33",
    'http://www.finance.gov.au/publications/parliamentarians-reporting/parliamentarians_expenditure_P32.html':"32",
    "http://www.finance.gov.au/publications/parliamentarians-reporting/parliamentarians_expenditure_P31.html":"31",
    "http://www.finance.gov.au/publications/parliamentarians-reporting/parliamentarians_expenditure_T30.html":"30"   
}    

for url in urls.keys():
    rep = requests.get(url)
    soup = BeautifulSoup(rep.content)
    links = soup.find_all("a")
    for l in links:
        try:
            if "pdf" in l.get("href"):
                pdfs.append(l.get("href"))
        except:
            pass

for p in pdfs:

    downloaded = !cd pdfs:ls
    local_filename = p.split('/')[-1]
    if "30" in p:
        local_filename = "P30-"+local_filename
    if "31" in url:
        local_filename = "P31-"+local_filename
    if local_filename not in downloaded:
        print 'hello',local_filename
    else:
        print 'other',local_filename


hello P34_BRODTMANN_Gai.pdf
hello P34_HUMPHRIES_Gary.pdf
hello P34_LEIGH_Andrew.pdf
hello P34_LUNDY_Kate.pdf?v=1
hello P34_SESELJA_Zed.pdf
hello P34_ABBOTT_Tony.pdf
hello P34_ALBANESE_Anthony.pdf
hello P34_ALEXANDER_John.pdf
hello P34_BALDWIN_Bob.pdf
hello P34_BIRD_Sharon.pdf
hello P34_BISHOP_Bronwyn.pdf
hello P34_BOWEN_Chris.pdf
hello P34_BURKE_Tony.pdf
hello P34_CAMERON_Doug.pdf
hello P34_CARR_Bob.pdf
hello P34_CLARE_Jason.pdf
hello P34_CLAYDON_Sharon.pdf
hello P34_COBB_John.pdf
hello P34_COLEMAN_David.pdf
hello P34_COMBET_Greg.pdf
hello P34_CONROY_Pat.pdf
hello P34_COONAN_Helen.pdf
hello P34_COULTON_Mark.pdf
hello P34_DASTYARI_Sam.pdf
hello P34_ELLIOT_Justine.pdf
hello P34_FAULKNER_John.pdf
hello P34_FERGUSON_Laurie.pdf
hello P34_FIERRAVANTI-WELLS_Concetta.pdf
hello P34_FITZGIBBON_Joel.pdf
hello P34_FLETCHER_Paul.pdf
hello P34_GASH_Joanna.pdf
hello P34_GILLESPIE_David.pdf
hello P34_GRIERSON_Sharon.pdf
hello P34_HALL_Jill.pdf
hello P34_HARTSUYKER_Luke.pdf
hello P34_HAWKE_Alex.pdf
hel

In [7]:
# Next, let's check about all potential categories by looking at what terms are on the summary page - for all pdfs.

sumref = {} # This is to find the summary reference page for all pdfs

localpdfs = !cd pdfs;ls # Gets the filenames

import pdfquery

def get_page(x,y): # this takes the pdfquery object and a list of pages to test (including None as catchall)
    for page in y:
        if page != None:
            x.load(page)
            titles = x.pq(":in_bbox('170.979,683.267,438.506,699.868')")
            for t in titles:
                if t.text.strip() == 'Summary of Parliamentary Expenditure by Period':
                    return int(page) # This returns the page
        else:
            x.load()
            tree = x.get_tree() # This is a last resort - search (and load!) the whole pdf to search for the summary
            for element in tree.iter():
                if element.text != '':
                    if element.text != None:
                        if element.text.strip() == 'Summary of Parliamentary Expenditure by Period':
                            for part in element.iterancestors(tag="LTPage"):
                                pageref = part.get("page_index")
                                return int(pageref)


# First, let's find the summary page for everyone
for pdffile in localpdfs:
    pdf = pdfquery.PDFQuery("pdfs/"+pdffile)
    sumref[pdffile] = get_page(pdf,[2,8,5,None]) # The most common pages are 2,8 and 5 - save time by trying these first

In [10]:
# This checks whether the total expenditure and the items added up are equal to each other.

import ast

import pandas as pd
from IPython.display import display
import collections                                         

def get_sum(x,y): #This takes the pdf and the page reference and produces summary stats
    refs = {}
    data = []

    colcounter = []
    colrefs = set()
    coldata = {}
    
   
    summary = x.get_tree(y)
    
    count = 0
    for element in summary.iter(): # this gets the name. It's the second text element on the page, so we need a count.
        if element.text != None:
            if element.text != '':
                count += 1
                if count == 2:
                    nom = element.text.strip()
                    break
                
    for element in summary.iter(): # This checks where the family expenses kick in
        if element.tag != "pdfxml":
            if element.text != '':
                if ast.literal_eval(element.get("bbox"))[0] in [39.146,52.106]:
                    refs[ast.literal_eval(element.get("bbox"))[1]] = element.text.strip()
                    if "Family Travel Costs" == element.text.strip():
                        family = ast.literal_eval(element.get("bbox"))[1]
                #print element.text,ast.literal_eval(element.get("bbox"))
    #print 'refs',refs
    
    # This gets the column positions for the data
    for element in summary.iter():
        if element.tag != "pdfxml":
            if element.text != '':
                if ast.literal_eval(element.get("bbox"))[1] in refs.keys():
                    if '$' in element.text:
                        colcounter.append(int(ast.literal_eval(element.get("bbox"))[2]))
                        #data[refs[ast.literal_eval(element.get("bbox"))[1]]] = element.text
    
    #print 'colcounter',colcounter
    counter = collections.Counter(colcounter)
    for group in counter.most_common():
        if group[1] > 5: # This removes footnotes stuffing up the column alignment
            colrefs.update([group[0]])
    #print 'colrefs',colrefs
    
    # This creates a string for each data column                    
    for ref in colrefs:
        coldata[ref] = ''

    # This creates the string for each column
    for element in summary.iter():
        if element.tag != "pdfxml":
            if element.text != '':
                for ref in colrefs:
                    if -1 < ref-ast.literal_eval(element.get("bbox"))[2] < 1:
                        if '$' not in element.text:
                            coldata[ref]+=element.text
    #print 'coldata',coldata
    
    # This gets the location of the Family Travel Costs divider and appends Family Travel Costs to 
    # Line items below, excluding Total Expenditure
    for r in refs.keys():
        if r < family:
            if refs[r] != 'Total Expenditure':
                refs[r] = refs[r] + ' - Family'                        

    # This gets the data
    for element in summary.iter():
        if element.tag != "pdfxml":
            if element.text != '':
                for ref in refs.keys():
                    if ast.literal_eval(element.get("bbox"))[1] == ref:
                        for col in coldata.keys():
                            if -1<ast.literal_eval(element.get("bbox"))[2]-col<30: # This checks to see if the data bounding box matches up to the column (with some fat to allow for footnotes)
                                if " " in element.text:
                                    data.append([nom,refs[ref],coldata[col],float(element.text.replace("$","").replace(",","").split(" ")[0])])
                                else:
                                    data.append([nom,refs[ref],coldata[col],float(element.text.replace("$","").replace(",",""))])
    return data,coldata

bigdata = []

for name in sumref.keys():
    pdf = pdfquery.PDFQuery("pdfs/"+name)
    result = get_sum(pdf,sumref[name])
    for part in result[0]:
        bigdata.append(part)
    #df = pd.DataFrame(part,columns=['Name','Type','Period','Amount'])
    # Testing expenditure tally
    #for period in result[1].values():
    #    if not -0.01 < round(df[(df['Type']!='Total Expenditure')&(df['Period']!=period)]['Amount'].sum(),2) - round(df[(df['Type']=='Total Expenditure')&(df['Period']!=period)]['Amount'].sum(),2) < 0.01:
    #        print name,period
df = pd.DataFrame(bigdata,columns=['Name','Type','Period','Amount'])


ValueError: Shape of passed values is (1, 4), indices imply (4, 4)

In [95]:
df['Name']

0     <LTTextBoxHorizontal bbox="[44.536, 518.031, 3...
1     <LTTextBoxHorizontal bbox="[44.536, 518.031, 3...
2     <LTTextBoxHorizontal bbox="[44.536, 518.031, 3...
3     <LTTextBoxHorizontal bbox="[44.536, 518.031, 3...
4     <LTTextBoxHorizontal bbox="[44.536, 518.031, 3...
5     <LTTextBoxHorizontal bbox="[44.536, 518.031, 3...
6     <LTTextBoxHorizontal bbox="[44.536, 518.031, 3...
7     <LTTextBoxHorizontal bbox="[44.536, 518.031, 3...
8     <LTTextBoxHorizontal bbox="[44.536, 518.031, 3...
9     <LTTextBoxHorizontal bbox="[44.536, 518.031, 3...
10    <LTTextBoxHorizontal bbox="[44.536, 518.031, 3...
11    <LTTextBoxHorizontal bbox="[44.536, 518.031, 3...
12    <LTTextBoxHorizontal bbox="[44.536, 518.031, 3...
13    <LTTextBoxHorizontal bbox="[44.536, 518.031, 3...
14    <LTTextBoxHorizontal bbox="[44.536, 518.031, 3...
15    <LTTextBoxHorizontal bbox="[44.536, 518.031, 3...
16    <LTTextBoxHorizontal bbox="[44.536, 518.031, 3...
17    <LTTextBoxHorizontal bbox="[44.536, 518.03

In [76]:
# error in Melissa Park - it overstates her expenses
for name in ['P35_PARKE_Melissa.pdf']:
    pdf = pdfquery.PDFQuery("pdfs/"+name)
    result = get_sum(pdf,sumref[name])

    df = pd.DataFrame(result[0],columns=['Type','Period','Amount'])
    display(df)
    # Testing expenditure tally
    for period in result[1].values():
        print round(df[(df['Type']!='Total Expenditure')&(df['Period']!=period)]['Amount'].sum(),2)
        print round(df[(df['Type']=='Total Expenditure')&(df['Period']!=period)]['Amount'].sum(),2)
        if not -0.01 < round(df[(df['Type']!='Total Expenditure')&(df['Period']!=period)]['Amount'].sum(),2) - round(df[(df['Type']=='Total Expenditure')&(df['Period']!=period)]['Amount'].sum(),2) < 0.01:
            print name,period

,Type,Period,Amount
0,Parliamentary Travelling Allowance,Expenses From 1 July 2014,7148.00
1,Fares,Expenses From 1 July 2014,27286.76
2,Private-Plated Vehicle,Expenses From 1 July 2014,5010.98
3,COMCAR,Expenses From 1 July 2014,1398.58
4,Cabcharge,Expenses From 1 July 2014,40.87
5,Other Car Costs,Expenses From 1 July 2014,0.00
6,Office Consumables and Services,Expenses From 1 July 2014,5702.22
7,Printing and Communications,Expenses From 1 July 2014,25704.41
8,Publications,Expenses From 1 July 2014,1523.99
9,Telecommunications - Usage,Expenses From 1 July 2014,3020.24


129403.52
134414.5
P35_PARKE_Melissa.pdf Expenses Before 1 July 2014 
15418.08
15613.07
P35_PARKE_Melissa.pdf Expenses From 1 July 2014 


In [107]:
for name in ['P35_PARKE_Melissa.pdf']:
    pdf = pdfquery.PDFQuery("pdfs/"+name)
    tree = pdf.get_tree(2)
    for element in tree.iter():
        if element.text != None:
            if element.text != '':
                print element.text

1 July to 31 December 2014 
The Hon Melissa Parke MP 
Summary of Parliamentary Expenditure by Period 
Expenses From 
1 July 2014 
Expenses Before 
1 July 2014 
Parliamentary Travelling Allowance 
Fares 
Private-Plated Vehicle 
COMCAR 
Cabcharge 
Other Car Costs 
Office Consumables and Services 
Printing and Communications 
Publications 
Telecommunications - Usage 
Domestic Scheduled Fares 
Total Expenditure 
1 Includes an adjustment of -$25.00 
2 Includes an adjustment of -$25.00 
$7,148.00 
$27,286.76 
1 
$5,010.98 
$1,398.58 
$40.87 
$0.00 
$5,702.22 
$25,704.41 
$1,523.99 
$3,020.24 
$3,341.00 
$3,203.69 
$194.99 2 
$202.00 
$60.44 
$514.03 
$250.98 
$3,556.07 
$597.44 
$0.00 
$0.00 
$134,414.50 
$1,521.08 
$15,613.07 
Travelling Allowance 
Overseas Travel 
$0.00 
$0.00 
Domestic Scheduled Fares 
Charter 
$0.00 
$0.00 
Car Costs 
Office Facilities 
$52,567.47 
$1,976.36 
Office Fit Outs 
$0.00 
$0.00 
Office Administrative Costs 
Telecommunications 
Family Travel Costs 
Page 2 of 22

In [104]:
for element in tree.iter():
    if element.text != None:
        if element.text != '':
            print element.text
            break

The Hon Melissa Parke MP 


In [177]:
# Let's get some values about the doc. This will also give us information about what information to expect.

### First, check what kind of expenses are in there, and whether there are family expenses, 
# and return what pages they are on.

# These are our important categories:
# Travelling Allowance / Parliamentary Travelling Allowance
# Overseas Travel
# Domestic Scheduled Fares / Fares
# Car Costs / Private-Plated Vehicle
# Car Costs / COMCAR
# Car Costs / Cabcharge
# Office Administrative Costs / Office Consumables and Services
# Office Administrative Costs / Printing and Communications
# Office Administrative Costs / Publications
# Telecommunications / Telecommunications - Usage
# Telecommunications / Telecommunications - Residential - Official
# Family Travel Costs / various

cats = [
    "Travelling Allowance / Parliamentary Travelling Allowance",
"Overseas Travel",
"Domestic Scheduled Fares / Fares",
"Car Costs / Private-Plated Vehicle",
"Car Costs / COMCAR",
"Car Costs / Cabcharge",
"Office Administrative Costs / Office Consumables and Services",
"Office Administrative Costs / Printing and Communications",
"Office Administrative Costs / Publications",
"Telecommunications / Telecommunications - Usage",
"Telecommunications / Telecommunications - Residential - Official",
"Family Travel Costs / various",
"Summary of Parliamentary Expenditure by Period / "
]


import ast

pagerefs = {} # This is to record the page refs for the various categories

dics = {} # This holds all our data



for i,element in enumerate(tree.iter()):
    
    # This runs through each element...
    if element.tag != "pdfxml": # excludes the first container element
        if element.text != '': # excludes empty elements
            if element.text != None: # excludes other empty elements
                for c in cats: # This finds what page each of the important categories is on
                    if c.split(" / ")[0] in element.text.strip():
                        for pageno in element.iterancestors(tag="LTPage"):
                            pagerefs.setdefault(c.split(" / ")[0], set()).update([int(pageno.get("page_index"))])

    # This part repeats the same as above, but is for collecting the data.
    if element.tag != "pdfxml": # excludes the first container element
        if element.text != '': # excludes empty elements
            if element.text != None: # excludes other empty elements
                # This part adds all the data elements to the dics disctionary is they line up horizontally.
                #if round(ast.literal_eval(element.get("bbox"))[1],1) in dics[page].keys():
                    #print page,element.text,element.get("bbox")
                #    dics[page][round(ast.literal_eval(element.get("bbox"))[1],1)].append(element.text)
                #else:
                    #print page,element.text,element.get("bbox")
                #    dics[page][round(ast.literal_eval(element.get("bbox"))[1],1)] = [element.text]
                pass

pagerefs

{'Car Costs': {1, 2, 4, 10, 11, 12, 13, 14, 23, 26},
 'Domestic Scheduled Fares': {2, 4, 8, 9, 21, 23, 25, 30},
 'Family Travel Costs': {2, 4, 21, 23, 30},
 'Office Administrative Costs': {2, 4, 17, 18, 19, 23, 28},
 'Overseas Travel': {2, 4, 6, 7},
 'Summary of Parliamentary Expenditure by Period': {2},
 'Telecommunications': {1, 2, 4, 20, 23, 29},
 'Travelling Allowance': {2, 4, 5, 23, 24}}

Expenditure checks out?
----
EXPENSES BEFORE 1 JULY 2014 
Item tally: 193815.97
Total     : 193815.97
-----
EXPENSES FROM 1 JULY 2014 
Item tally: 15039.78
Total     : 15039.78
-----


In [168]:
pagerefs

{'Car Costs': {'1', '10', '11', '12', '13', '14', '2', '23', '26', '4'},
 'Domestic Scheduled Fares': {'2', '21', '23', '25', '30', '4', '8', '9'},
 'Family Travel Costs': {'2', '21', '23', '30', '4'},
 'Office Administrative Costs': {'17', '18', '19', '2', '23', '28', '4'},
 'Overseas Travel': {'2', '4', '6', '7'},
 'Summary of Parliamentary Expenditure by Period': {'2'},
 'Telecommunications': {'1', '2', '20', '23', '29', '4'},
 'Travelling Allowance': {'2', '23', '24', '4', '5'}}

In [185]:
# Third, let's get how many expenses are in each category.

for cat in pagerefs.keys()[:1]:
    print cat
    for page in pagerefs[cat]:
        if int(page) > 4: # this ignores the first four pages - summary and cover pages
            print page
            tree = pdf.get_tree(page)
            for element in tree.iter():
                if element.text != None:
                    if element.text != '':
                        if element.text.strip() == 'Family':
                            print 'family',ast.literal_eval(element.get('bbox'))
                        if element.text.strip() == 'Private-Plated Vehicle':
                            print 'pp',ast.literal_eval(element.get('bbox'))
                        if element.text.strip() == 'COMCAR':
                            print 'cc',ast.literal_eval(element.get('bbox'))
                        if element.text.strip() == 'Cabcharge':
                            print 'cab',ast.literal_eval(element.get('bbox'))
         
            


Car Costs
10
pp [30.362, 673.077, 140.462, 688.401]
cc [30.362, 297.669, 75.662, 312.993]
11
cc [30.362, 673.077, 75.662, 688.401]
12
cc [30.362, 673.077, 75.662, 688.401]
13
cc [30.362, 673.077, 75.662, 688.401]
cab [30.362, 145.533, 82.406, 160.857]
14
cab [30.362, 673.077, 82.406, 688.401]
23
pp [80.618, 497.177, 169.938, 509.887]
cc [80.618, 480.185, 117.638, 492.895]
cab [80.618, 463.193, 123.398, 475.903]
26
pp [30.362, 673.077, 140.462, 688.401]
cc [30.362, 531.525, 75.662, 546.849]
cab [30.362, 389.973, 82.406, 405.297]


In [ ]:


# Fourt, let's compare the expenses with the totals.

In [18]:
import ast

page = 0
dics = {}
for i,element in enumerate(tree.iter()):
    if element.tag == "LTPage":
        page = element.get("page_index")
        dics[page]={}
    if element.tag != "pdfxml":
        if element.text != '':
            if element.text != None:
                if round(ast.literal_eval(element.get("bbox"))[1],1) in dics[page].keys():
                    print page,element.text,element.get("bbox")
                    dics[page][round(ast.literal_eval(element.get("bbox"))[1],1)].append(element.text)
                else:
                    print page,element.text,element.get("bbox")
                    dics[page][round(ast.literal_eval(element.get("bbox"))[1],1)] = [element.text]

0 Ms Gai Brodtmann MP  [44.536, 518.031, 266.176, 548.535]
0 Expenditure on Entitlements paid by the  [44.536, 450.271, 372.136, 475.691]
0 Department of Finance  [44.536, 425.863, 231.976, 451.283]
0 1 July to 31 December 2014  [44.536, 377.695, 267.256, 403.495]
0 Final Report  [44.536, 306.843, 135.13, 329.757]
1 1 July to 31 December 2014  [31.658, 567.114, 145.668, 579.884]
1 Ms Gai Brodtmann MP  [31.658, 542.998, 142.478, 558.25]
1 Summary of Parliamentary Expenditure  [278.403, 503.011, 508.143, 520.889]
1 Paid between 1 July and 31 December 2014  [294.243, 483.961, 492.386, 498.008]
1 Ms Gai Brodtmann MP  [31.226, 402.775, 105.106, 412.943]
1 Canberra  [154.059, 402.775, 183.931, 412.943]
1 $708.00  [237.003, 403.313, 260.019, 412.21]
1 $0.00  [289.995, 403.313, 305.927, 412.21]
1 $2,771.03  [335.211, 403.313, 363.512, 412.21]
1 $0.00  [388.995, 403.313, 404.927, 412.21]
1 $7,131.82  [433.059, 403.313, 461.36, 412.21]
1 $66,530.61  [481.011, 403.313, 512.854, 412.21]
1 $18,142.

In [94]:
x = 2

1<x<2.1

True

SelectorSyntaxError: Expected selector, got <DELIM '/' at 0>

In [50]:
x0,y0,x1,y1

'LTTextLineHorizontal'

In [47]:
y.

<Element LTTextLineHorizontal at 0x107b5faa0>

In [114]:
df.groupby('Type').sum()['Amount']

Type
Additional Printing and Communications             2423.19
COMCAR                                          1637429.17
COMCAR - Family                                   32309.36
Cabcharge                                         55918.72
Cabcharge - Family                                  138.55
Car Costs                                             0.00
Charter                                               0.00
Daily Expense Allowance                            9102.00
Domestic Scheduled Fares                              0.00
Domestic Scheduled Fares - Family                998265.87
Electorate                                       320327.86
Family Travel Costs                                   0.00
Fares                                           6468418.72
Ministerial Visits                              2678365.28
Office Administrative Costs                           0.00
Office Consumables and Services                 2742120.88
Office Facilities                              1643

In [128]:
for cat in df['Type'].unique():
    display(cat,df[df['Type'] == cat].groupby('Name').sum().sort('Amount',ascending=False)[:3])

/Applications/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:2: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)
  from ipykernel import kernelapp as app


'Parliamentary Travelling Allowance'

,Amount
Name,
The Hon Warren Snowdon MP,43615
The Hon Warren Truss MP,36930
Senator the Hon Nigel Scullion,36348


'Fares'

,Amount
Name,
Senator Rachel Siewert,89849.19
Senator the Hon Mathias Cormann,84984.66
The Hon Michael Keenan MP,79415.42


...

,Amount
Name,
The Hon Bob Baldwin MP,1480.00
Senator the Hon Ian Macdonald,122.58


'SMOS Approved'

,Amount
Name,
Mr Mark Coulton MP,35181.80
The Hon Darren Chester MP,15714.76
The Hon Alan Tudge MP,3027.27


'Office Holder'

,Amount
Name,
The Hon Barnaby Joyce MP,75807.27
The Hon Warren Truss MP,67117.27
The Hon Ian Macfarlane MP,50410.64


In [130]:
df[df['Type'] == 'SMOS Approved']

,Name,Type,Period,Amount
1496,The Hon Alan Tudge MP,SMOS Approved,Expenses From 1 July 2014,0.00
1497,The Hon Alan Tudge MP,SMOS Approved,Expenses Before 1 July 2014,3027.27
7008,The Hon Michael McCormack MP,SMOS Approved,Expenses From 1 July 2014,2590.00
7024,The Hon Michael McCormack MP,SMOS Approved,Expenses Before 1 July 2014,0.00
7440,The Hon Darren Chester MP,SMOS Approved,Expenses From 1 July 2014,10804.69
7454,The Hon Darren Chester MP,SMOS Approved,Expenses Before 1 July 2014,4910.07
7804,Mr Mark Coulton MP,SMOS Approved,Expenses From 1 July 2014,28272.70
7820,Mr Mark Coulton MP,SMOS Approved,Expenses Before 1 July 2014,6909.10
